Notebook walking through constructing a Dataset with DatasetBuilder

In [1]:
import sys
import pandas as pd
sys.path.append("..")

from chemspace.Dataset.DatasetBuilder import DatasetBuilder

Dataset Builder Class can be instantiated one of 3 ways:
1. From a Chemical Structures Records `.json.gz` file downloaded from PubChem's search function
2. From a `.csv` file containing CIDs as the indices of the file
3. From a previously constructed DataFrame that has CIDs of interest as the indices

Method 1: Instantiate a DatasetBuilder object from a Chemical Structures Records `.json.gz` file downloaded from PubChem's search function

In [ ]:
# Instantiate class with json file form PubChem
DB = DatasetBuilder(compound_file_path='../chemspace/Dataset/Data/PubChem_compound_list_records.json.gz')

# Display dataset
DB.dataset

# Save as .CSV
#DB.CIDs.to_csv('../chemspace/Dataset/Data/CIDs.csv', index = True)

Method 2: Instantiate a DatasetBuilder object from a `.csv` file containing CIDs as the indices of the file

In [2]:
# Instantiate class with previously generated Dataframe (CSV)
DB = DatasetBuilder(compound_file_path='../chemspace/Dataset/Data/CIDs.csv')

# Display dataset
DB.CIDs

""
1
3
4
5
6
...
168010365
168010381
168010385
168010395


Method 3: Instantiate a DatasetBuilder object from a previously constructed DataFrame that has CIDs of interest as the indices

In [ ]:
# Load df
df = pd.read_csv('../chemspace/Dataset/Data/CIDs.csv', index_col='Unnamed: 0')

# Instantiate class with previously generated Dataframe
DB = DatasetBuilder(compound_df=df)

# Display dataset
DB.CIDs

In [3]:
DB.add_SMILES()

0
5


In [ ]:
print(len(DB.dataset.index))
print(len(DB.CIDs))

In [ ]:
DB.dataset

In [ ]:
DB.CIDs.to_csv('../chemspace/Dataset/Data/CIDs.csv', index = True)